# complib.base_component

> TODO fill in description

In [ ]:
#| default_exp complib.exec_component

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide®
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from abc import abstractmethod

import fbdev
from fbdev.comp.packet import Packet
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection
from fbdev.comp.base_component import BaseComponent

In [ ]:
#|hide
show_doc(fbdev.complib.ExecComponent)

---

### ExecComponent

>      ExecComponent ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class ExecComponent(BaseComponent):
    execute_after_start = True
    loop_execution = False
    
    port_specs = PortSpecCollection(
        PortSpec(PortType.SIGNAL, 'execute'),
        PortSpec(PortType.MESSAGE, 'executed')
    )
    
    def __init__(self):
        super().__init__()
        self._main_task: asyncio.Task = None
    
    async def _post_start(self):
        self.task_manager.create_task(self._pre_execute())
        if self.execute_after_start:
            self.task_manager.create_task(
                self.ports[(PortType.SIGNAL, 'execute')]._put(Packet.get_empty())
            )
        self._main_task = self.task_manager.create_task(self._pre_execute())
    
    async def _pre_execute(self):
        if not self.loop_execution:
            await self.await_signal('execute')
        await self.update_config()
        await self._execute()
        await self.send_message('executed')
        self._main_task = self.task_manager.create_task(self._pre_execute())
    
    @abstractmethod
    async def _execute(self):
        raise NotImplementedError()

In [ ]:
class MyExecComponent(ExecComponent):
    async def _execute(self):
        print("I'm executing!")
    
comp_process = MyExecComponent()
await comp_process.start()
await comp_process.ports[(PortType.MESSAGE, 'executed')]._get()
await comp_process.stop()

I'm executing!
